In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

from plotly import graph_objects as go
import plotly as py

from datetime import datetime
from datetime import timedelta

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
df_merged = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/merged.csv', sep = ';')
df_owm = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/OpenWeatherMapSantoAndre.csv')

In [4]:
df_merged['Data_Hora'] = pd.to_datetime(df_merged['Data_Hora'])
df_owm['Data_Hora'] = pd.to_datetime(df_owm['dt_iso'].str[:-10])
df_owm['Data_Hora'] = df_owm['Data_Hora'] - pd.Timedelta(hours = 3)
df_owm_f = df_owm[df_owm['Data_Hora'] >= datetime.strptime('2010-01-01', '%Y-%m-%d')]
df_owm_f

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,Data_Hora
274776,1262314800,2010-01-01 03:00:00 +0000 UTC,-7200,Santo Andre,-23.674223,-46.5436,21.39,22.40,20.82,24.01,1015,NaN,NaN,94,4.1,140,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n,2010-01-01 00:00:00
274777,1262318400,2010-01-01 04:00:00 +0000 UTC,-7200,Santo Andre,-23.674223,-46.5436,21.28,21.74,20.48,23.80,1014,NaN,NaN,88,4.1,160,0.35,NaN,NaN,NaN,75,500,Rain,light rain,10n,2010-01-01 01:00:00
274778,1262322000,2010-01-01 05:00:00 +0000 UTC,-7200,Santo Andre,-23.674223,-46.5436,21.29,22.80,20.55,23.78,1013,NaN,NaN,88,2.6,160,0.80,NaN,NaN,NaN,75,500,Rain,light rain,10n,2010-01-01 02:00:00
274779,1262325600,2010-01-01 06:00:00 +0000 UTC,-7200,Santo Andre,-23.674223,-46.5436,21.13,22.63,20.50,24.07,1012,NaN,NaN,93,3.1,160,1.79,NaN,NaN,NaN,75,500,Rain,light rain,10n,2010-01-01 03:00:00
274780,1262329200,2010-01-01 07:00:00 +0000 UTC,-7200,Santo Andre,-23.674223,-46.5436,21.25,22.89,20.49,24.08,1012,NaN,NaN,94,3.1,180,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370034,1597518000,2020-08-15 19:00:00 +0000 UTC,-10800,Santo Andre,-23.674223,-46.5436,21.10,20.57,19.00,23.77,1015,NaN,NaN,77,4.1,160,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04d,2020-08-15 16:00:00
370035,1597521600,2020-08-15 20:00:00 +0000 UTC,-10800,Santo Andre,-23.674223,-46.5436,20.49,20.47,18.00,22.95,1015,NaN,NaN,82,3.6,140,NaN,NaN,NaN,NaN,90,701,Mist,mist,50d,2020-08-15 17:00:00
370036,1597525200,2020-08-15 21:00:00 +0000 UTC,-10800,Santo Andre,-23.674223,-46.5436,20.28,19.13,18.00,22.95,1015,NaN,NaN,82,5.1,160,NaN,NaN,NaN,NaN,90,701,Mist,mist,50n,2020-08-15 18:00:00
370037,1597528800,2020-08-15 22:00:00 +0000 UTC,-10800,Santo Andre,-23.674223,-46.5436,19.47,19.50,17.00,22.95,1016,NaN,NaN,88,3.6,160,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n,2020-08-15 19:00:00


In [5]:
df_owm.isna().sum()

dt                          0
dt_iso                      0
timezone                    0
city_name                   0
lat                         0
lon                         0
temp                        0
feels_like                  0
temp_min                    0
temp_max                    0
pressure                    0
sea_level              370039
grnd_level             370039
humidity                    0
wind_speed                  0
wind_deg                    0
rain_1h                320985
rain_3h                370035
snow_1h                370039
snow_3h                370039
clouds_all                  0
weather_id                  0
weather_main                0
weather_description         0
weather_icon                0
Data_Hora                   0
dtype: int64

In [6]:
df_owm_f.loc[~df_owm_f['rain_1h'].isna(), ['Data_Hora', 'rain_1h', 'weather_description']]

,Data_Hora,rain_1h,weather_description
274777,2010-01-01 01:00:00,0.35,light rain
274778,2010-01-01 02:00:00,0.80,light rain
274779,2010-01-01 03:00:00,1.79,light rain
274783,2010-01-01 07:00:00,0.80,light rain
274784,2010-01-01 07:00:00,0.80,mist
...,...,...,...
370025,2020-08-15 08:00:00,0.33,light rain
370026,2020-08-15 09:00:00,0.33,light rain
370029,2020-08-15 12:00:00,0.06,light rain
370030,2020-08-15 12:00:00,0.06,mist


In [7]:
df = df_merged.merge(df_owm_f, on = 'Data_Hora')
df.loc[~df['rain_1h'].isna(), ['Data_Hora'] + [c for c in df.columns if 'Precipitacao' in c] + ['rain_1h']].corr()

,Precipitacao_0,Precipitacao_1,Precipitacao_2,Precipitacao_3,Precipitacao_4,rain_1h
Precipitacao_0,1.000000,0.584687,0.508766,0.544399,0.563863,0.094855
Precipitacao_1,0.584687,1.000000,0.401332,0.443080,0.464892,0.054120
Precipitacao_2,0.508766,0.401332,1.000000,0.712313,0.675332,0.112217
Precipitacao_3,0.544399,0.443080,0.712313,1.000000,0.630107,0.117747
Precipitacao_4,0.563863,0.464892,0.675332,0.630107,1.000000,0.098628
rain_1h,0.094855,0.054120,0.112217,0.117747,0.098628,1.000000


In [8]:
df['Data'] = df['Data_Hora'].dt.strftime('%Y-%m-%d')
df.loc[~df['rain_1h'].isna(), ['Data'] + [c for c in df.columns if 'Precipitacao' in c] + ['rain_1h']].groupby('Data').mean()

,Precipitacao_0,Precipitacao_1,Precipitacao_2,Precipitacao_3,Precipitacao_4,rain_1h
Data,,,,,,
2011-01-02,2.415385,2.515385,1.846154,2.069231,0.0,2.100769
2011-01-03,0.981818,0.954545,0.863636,0.827273,0.0,2.016818
2011-01-04,1.466667,1.600000,1.200000,2.700000,0.0,2.208333
2011-01-05,0.216667,0.250000,0.116667,0.216667,0.0,0.356667
2011-01-06,2.450000,2.200000,3.700000,3.650000,0.0,0.457500
...,...,...,...,...,...,...
2019-09-12,0.000000,NaN,0.000000,0.000000,0.0,0.150000
2019-09-20,0.100000,NaN,0.200000,0.400000,0.0,0.320000
2019-09-22,0.044444,NaN,0.266667,0.177778,0.0,1.050000


In [9]:
df['Data_Hora'].dt.strftime('%Y-%m-%d')

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
78185    2015-07-16
78186    2015-07-17
78187    2015-07-17
78188    2015-07-17
78189    2015-07-17
Name: Data_Hora, Length: 78190, dtype: object

# blablabla

In [17]:
df_est = pd.read_csv('../../../ordens.csv', sep=';')
df_est['Data'] = pd.to_datetime(df_est['Data'], yearfirst=True)
df_est.head()

,Data,Vitoria,Erasmo,Paraiso,Null,RM,Camilopolis
0,2010-01-07,21,4,0,0,0,0
1,2010-01-08,11,0,0,0,0,0
2,2010-01-11,1,1,0,0,0,0
3,2010-01-19,3,0,2,1,0,0
4,2010-01-20,11,0,0,1,0,0


In [18]:
print(df_est['Data'].min())
print(df_est['Data'].max())

2010-01-07 00:00:00
2019-08-30 00:00:00


In [25]:
#df_owm.isna().sum()
df_o = df_owm[(datetime.strptime('2019-08-30', '%Y-%m-%d') >= df_owm['Data_Hora']) & (df_owm['Data_Hora'] >= datetime.strptime('2010-01-01', '%Y-%m-%d'))].copy()
df_o = df_o.drop(columns = ['sea_level', 'grnd_level', 'rain_3h', 'snow_1h', 'snow_3h'])
df_o = df_o.fillna(0)

In [29]:
df_o['Data_Hora'].unique().shape

(84673,)

In [30]:
df_owm.apply(lambda x: x['Data_Hora'] + pd.Timedelta(hours = x['timezone'] / 3600), axis = 1)

0        1978-12-31 18:00:00
1        1978-12-31 19:00:00
2        1978-12-31 20:00:00
3        1978-12-31 21:00:00
4        1978-12-31 22:00:00
                 ...        
370034   2020-08-15 13:00:00
370035   2020-08-15 14:00:00
370036   2020-08-15 15:00:00
370037   2020-08-15 16:00:00
370038   2020-08-15 17:00:00
Length: 370039, dtype: datetime64[ns]